In [0]:
#Model 2
#Importing dependencies
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import os
import datetime
import pickle
 
#Sets batch size (number of images inputted at one time) for training
batch_size = 1000
 
#Naming the model and adding a TensorBoard callback for graphs of metrics
NAME = "Malaria-Images-CNN-V1".format(int(time.time()))
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
 
#Loading in the data from Google Drive
pickle_in = open("/content/drive/My Drive/Science Fair/Malaria Project/X.pickle","rb")
X = pickle.load(pickle_in)
pickle_in = open("/content/drive/My Drive/Science Fair/Malaria Project/y.pickle","rb")
y = pickle.load(pickle_in)
X = np.array(X)
y = np.array(y)
X = X/255.0
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42) #Splits data into train and test
 
#Defining the model
model = Sequential()
 
#Adding the layers in the model
#First convolution and pooling block
model.add(Conv2D(128, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
#Second convolution and pooling block
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
#Flattens the stacks of features down
model.add(Flatten())
 
#Fully connected (regular) layer
model.add(Dense(64))
model.add(Activation('relu'))
 
#Output layer
model.add(Dense(1))
model.add(Activation('sigmoid'))
 
#Compiling model & adding metrics (accuracy, precision, recall)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
 
#Training the model
model.fit(X_train, y_train,batch_size=batch_size, epochs=20, validation_split=0.3, callbacks=[tensorboard])
 
#Evaluating the model on unseen data
scores = model.evaluate(X_test, y_test, verbose=1)
print(scores)
print(model.metrics_names)